In [1]:
import pickle
import sys

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

from src.cfd import CFD
from src.dataset import X_TIME, TIME_STEP
from src.gauss_hist import get_gauss_stats
from src.models import optimal_model_builder_all_ch
from src.network_utils import train_model as _base_train_model, plot_difference_hist, plot_history, compare_results, \
    gaussian_kernel

PWD = '../../../..'
sys.path.append(PWD)

tf.get_logger().setLevel('ERROR')

In [2]:
N_JOBS = 16

DATASET_PATH = PWD + '/data/dataset/dataset_r354332.pkl'

PRINT_PCOV = False

N_BASELINE = 6

BATCH_SIZE = 8192
LOSS_WEIGHT = 1000

CFD_THRESHOLD_SPACE = np.arange(0.10, 0.20, 0.005)
OPTIMAL_CFD_THRESHOLDS_TRAIN_DATA_PATH = PWD + '/data/tmp/many_channels_tests/optimal_cfd_thresholds_r354332.pkl'

TRAIN_TEST_SPLIT_RANDOM_STATE = 42
TEST_SIZE = 0.2

In [3]:
with open(DATASET_PATH, 'rb') as file:
    dataset = pickle.load(file)

# Utils

In [4]:
def model_predict(model: tf.keras.Model, X: np.ndarray):
    Y_heatmap_pred = model.predict(X, batch_size=4096)

    y_pred = np.empty(Y_heatmap_pred.shape[0])
    for i, y in enumerate(Y_heatmap_pred):
        _, y_pred[i], _, _ = get_gauss_stats(X_TIME, y, a_0=1, std_0=1. * TIME_STEP)

    return y_pred

In [5]:
def train_model(model, name, train=False, verbose=1, batch_size=BATCH_SIZE, loss_weights=LOSS_WEIGHT):
    return _base_train_model(model, name, 'many_channels_tests/trained_on_all', [], [], [], [], 0.01,
                             train, 1, verbose, batch_size, 1, 1, 1, loss_weights, root=PWD + '/data')

# Optimal CFD thresholds

In [6]:
from src.cfd import find_optimal_cfd_threshold

optimal_cfd_thresholds = {}
for plane, channel in sorted(dataset.keys()):
    print(f'Processing p={plane}, ch={channel}...')
    all_X, all_y = dataset[(plane, channel)][0], dataset[(plane, channel)][1]
    X_train, _, y_train, _ = train_test_split(all_X, all_y, test_size=TEST_SIZE, random_state=TRAIN_TEST_SPLIT_RANDOM_STATE)

    optimal_cfd_threshold = find_optimal_cfd_threshold(CFD_THRESHOLD_SPACE, N_BASELINE, X_train, y_train, X_TIME, n_jobs=N_JOBS, plot=False)
    optimal_cfd_thresholds[(plane, channel)] = optimal_cfd_threshold

with open(OPTIMAL_CFD_THRESHOLDS_TRAIN_DATA_PATH, 'wb') as file:
    pickle.dump(optimal_cfd_thresholds, file)

Processing p=1, ch=2...
Optimal CFD threshold: 0.170
Processing p=1, ch=11...
Optimal CFD threshold: 0.165
Processing p=2, ch=2...
Optimal CFD threshold: 0.195
Processing p=2, ch=11...
Optimal CFD threshold: 0.145
Processing p=3, ch=2...
Optimal CFD threshold: 0.135
Processing p=3, ch=11...
Optimal CFD threshold: 0.140


In [7]:
with open(OPTIMAL_CFD_THRESHOLDS_TRAIN_DATA_PATH, 'rb') as file:
    optimal_cfd_thresholds = pickle.load(file)

print('Optimal thresholds:')
for (plane, channel), threshold in optimal_cfd_thresholds.items():
    print(f'({plane:>1}, {channel:>2}): {threshold:0.3f}')

Optimal thresholds:
(1,  2): 0.170
(1, 11): 0.165
(2,  2): 0.195
(2, 11): 0.145
(3,  2): 0.135
(3, 11): 0.140


# Load the network

In [8]:
model = optimal_model_builder_all_ch()
history = train_model(model, "optimal")

In [9]:
plot_history(history, f'model: optimal', ymax=20)

# Test

Processing p=1, ch=2...
1/1 [==============================] - 1s 848ms/step
       CFD: 81.80 ps
   optimal: 74.11 ps (improvement: 9.40 %)
Processing p=1, ch=11...
1/1 [==============================] - 0s 379ms/step
       CFD: 92.08 ps
   optimal: 85.01 ps (improvement: 7.67 %)
Processing p=2, ch=2...
1/1 [==============================] - 0s 315ms/step
       CFD: 86.38 ps
   optimal: 75.29 ps (improvement: 12.83 %)
Processing p=2, ch=11...
1/1 [==============================] - 0s 368ms/step
       CFD: 91.39 ps
   optimal: 86.59 ps (improvement: 5.25 %)
Processing p=3, ch=2...
1/1 [==============================] - 0s 311ms/step
       CFD: 99.12 ps
   optimal: 87.50 ps (improvement: 11.73 %)
Processing p=3, ch=11...
1/1 [==============================] - 0s 386ms/step
       CFD: 119.48 ps
   optimal: 104.88 ps (improvement: 12.22 %)


In [10]:
improvements = {}
for (plane, channel), (all_X, all_y) in dataset.items():
    print(f'Processing p={plane}, ch={channel}...')

    _, X_test, _, y_test = train_test_split(all_X, all_y, test_size=TEST_SIZE, random_state=TRAIN_TEST_SPLIT_RANDOM_STATE)

    cfd = CFD(n_baseline=N_BASELINE, threshold=optimal_cfd_thresholds[(plane, channel)])
    y_pred_cfd = cfd.predict(X_TIME, X_test)
    std_cfd, _, _ = plot_difference_hist(y_test, y_pred_cfd, show=False, close=False)

    y_pred_model = model_predict(model, X_test)
    std_model, _, _ = plot_difference_hist(y_test, y_pred_model, show=False, close=False)

    compare_results([std_model], ["optimal"], std_cfd)
    improvements[(plane, channel)] = (std_cfd, std_model, 1 - std_model / std_cfd)

In [11]:
print('Improvements:')
for (plane, channel), improv in improvements.items():
    print(
        f'({plane:>1}, {channel:>2}) CFD: {improv[0] * 1000:6.2f} ps, NN: {improv[1] * 1000:6.2f} ({improv[2] * 100:5.2f} %)')

Improvements:
(1,  2) CFD:  81.80 ps, NN:  74.11 ( 9.40 %)
(1, 11) CFD:  92.08 ps, NN:  85.01 ( 7.67 %)
(2,  2) CFD:  86.38 ps, NN:  75.29 (12.83 %)
(2, 11) CFD:  91.39 ps, NN:  86.59 ( 5.25 %)
(3,  2) CFD:  99.12 ps, NN:  87.50 (11.73 %)
(3, 11) CFD: 119.48 ps, NN: 104.88 (12.22 %)
